# Memory
* Save your conversation with the LLM.

## Intro
* Ability to store information about past interactions.
* **Most of memory-related functionality in LangChain is marked as beta**. This is for two reasons:
    1. Most functionality are not production ready.
    2. Most functionality work with Legacy chains, not the newer LCEL syntax.
* **The main exception to this is the ChatMessageHistory functionality**. This functionality is largely production ready and does integrate with LCEL.

## LangChain documentation on Memory
* See the LangChain documentation page on Memory [here](https://python.langchain.com/v0.1/docs/modules/memory/).
* See the LangChain documentation page on how to use ChatMessageHistory with LCEL [here](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/).
* See the LangChain documentation page on the various ChatMessageHistory integrations [here](https://python.langchain.com/v0.1/docs/integrations/memory/).

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 001-simple-chain.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **001-conversation-buffer**.

## Track operations
From now on, we can track the operations **and the cost** of this project from LangSmith:
* [smith.langchain.com](https://smith.langchain.com)

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [1]:
#pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

## Buffer Memory
* ConversationBufferMemory keeps a list of chat messages in a buffer and passes those into the prompt template.
* A buffer refers to a **temporary storage area in memory** used to hold data. Here, the buffer specifically holds chat messages before they are processed or used in some way, such as being passed into a prompt template.
* In simple terms, a buffer is like a waiting room for data. It's a temporary holding spot where data can stay until it's ready to be used or processed. Imagine you're cooking and you chop up vegetables before you need to cook them. Instead of chopping each vegetable right before it goes in the pan, you chop them all at once and put them aside on a cutting board. That cutting board with the chopped vegetables is like a buffer — it holds everything ready for you until you're ready to use it. This way, when you need the vegetables, they are all prepared and ready to go, making the cooking process smoother and faster.
* As you can see in the following example, ConversationBufferMemory was used mostly in the initial versions of LangChain.

In [6]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

C:\Users\Juan\AppData\Local\Temp\ipykernel_19212\131978567.py:23: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\Juan\AppData\Local\Temp\ipykernel_19212\131978567.py:25: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(


In [37]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation.invoke({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you today?
Human: My name is Julio and I have moved 33 times.
AI: Nice to meet you, Julio! That's quite a lot of moves. What made you move so many times?
Human: If my average moving distance was 100 miles, how many miles took all my moves?
AI: To calculate the total distance of all your moves, we can multiply the number of moves by the average distance per move.

Total distance = Number of moves x Average distance per move

Total distance = 33 moves x 100 miles = 3300 miles

So, all your moves totaled 3300 miles. That's quite a journey!
Human: Do you remember my name?
AI: Yes, Julio!
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Julio and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, Julio! That's quite a lot of moves. What made you move so many times?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="To calculate the total distance of all your moves, we can multiply the number of moves by the average distance per move.\n\nTotal distance = Number of moves x Average distance per move\n\nTotal distance = 33 moves x 100 miles = 3300 miles\n\nSo, all your moves totaled 3300 miles. That's quite a journey!", additional_kwargs={}, response_metadata={}),
  H

In [9]:
conversation({"question": "My name is Julio and I have moved 33 times."})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you today?
Human: My name is Julio and I have moved 33 times.

> Finished chain.


{'question': 'My name is Julio and I have moved 33 times.',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Julio and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, Julio! That's quite a lot of moves. What made you move so many times?", additional_kwargs={}, response_metadata={})],
 'text': "Nice to meet you, Julio! That's quite a lot of moves. What made you move so many times?"}

In [10]:
conversation({"question": "If my average moving distance was 100 miles, how many miles took all my moves?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you today?
Human: My name is Julio and I have moved 33 times.
AI: Nice to meet you, Julio! That's quite a lot of moves. What made you move so many times?
Human: If my average moving distance was 100 miles, how many miles took all my moves?

> Finished chain.


{'question': 'If my average moving distance was 100 miles, how many miles took all my moves?',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Julio and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, Julio! That's quite a lot of moves. What made you move so many times?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="To calculate the total distance of all your moves, we can multiply the number of moves by the average distance per move.\n\nTotal distance = Number of moves x Average distance per move\n\nTotal distance = 33 moves x 100 miles = 3300 miles\n\nSo, all your moves totaled 3300 miles

In [11]:
conversation({"question": "Do you remember my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you today?
Human: My name is Julio and I have moved 33 times.
AI: Nice to meet you, Julio! That's quite a lot of moves. What made you move so many times?
Human: If my average moving distance was 100 miles, how many miles took all my moves?
AI: To calculate the total distance of all your moves, we can multiply the number of moves by the average distance per move.

Total distance = Number of moves x Average distance per move

Total distance = 33 moves x 100 miles = 3300 miles

So, all your moves totaled 3300 miles. That's quite a journey!
Human: Do you remember my name?

> Finished chain.


{'question': 'Do you remember my name?',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Julio and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, Julio! That's quite a lot of moves. What made you move so many times?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="To calculate the total distance of all your moves, we can multiply the number of moves by the average distance per move.\n\nTotal distance = Number of moves x Average distance per move\n\nTotal distance = 33 moves x 100 miles = 3300 miles\n\nSo, all your moves totaled 3300 miles. That's quite a journey!", additional_kwargs={}, resp

In [12]:
print(memory.buffer)

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='My name is Julio and I have moved 33 times.', additional_kwargs={}, response_metadata={}), AIMessage(content="Nice to meet you, Julio! That's quite a lot of moves. What made you move so many times?", additional_kwargs={}, response_metadata={}), HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}), AIMessage(content="To calculate the total distance of all your moves, we can multiply the number of moves by the average distance per move.\n\nTotal distance = Number of moves x Average distance per move\n\nTotal distance = 33 moves x 100 miles = 3300 miles\n\nSo, all your moves totaled 3300 miles. That's quite a journey!", additional_kwargs={}, response_metadata={}), HumanMessage(content='Do you remember my name?', 

In [13]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Julio and I have moved 33 times.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Nice to meet you, Julio! That's quite a lot of moves. What made you move so many times?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='If my average moving distance was 100 miles, how many miles took all my moves?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="To calculate the total distance of all your moves, we can multiply the number of moves by the average distance per move.\n\nTotal distance = Number of moves x Average distance per move\n\nTotal distance = 33 moves x 100 miles = 3300 miles\n\nSo, all your moves totaled 3300 miles. That's quite a journey!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content

## Conversation Buffer Window Memory
Similar to the previous one, but **you can limit the number of conversational exchanges stored in memory**. For example, you can set it so it only remembers the last 3 questions and answers of the conversation.

In [14]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

In [15]:
window_memory = ConversationBufferWindowMemory(k=3)

C:\Users\Juan\AppData\Local\Temp\ipykernel_19212\3645135692.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  window_memory = ConversationBufferWindowMemory(k=3)


In [16]:
conversation_window = ConversationChain(
    llm=llm, 
    memory = window_memory,
    verbose=True
)

C:\Users\Juan\AppData\Local\Temp\ipykernel_19212\2450156885.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_window = ConversationChain(


In [17]:
conversation_window({"input": "Hi, my name is Julio"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Julio
AI:

> Finished chain.


{'input': 'Hi, my name is Julio',
 'history': '',
 'response': "Hello, Julio! It's nice to meet you. How can I assist you today?"}

In [18]:
conversation_window({"input": "My favorite color is blue"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Julio
AI: Hello, Julio! It's nice to meet you. How can I assist you today?
Human: My favorite color is blue
AI:

> Finished chain.


{'input': 'My favorite color is blue',
 'history': "Human: Hi, my name is Julio\nAI: Hello, Julio! It's nice to meet you. How can I assist you today?",
 'response': "Blue is a great color! It's often associated with calmness, stability, and trustworthiness. Do you have a specific reason why blue is your favorite color?"}

In [19]:
conversation_window({"input": "My favorite animals are dogs"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Julio
AI: Hello, Julio! It's nice to meet you. How can I assist you today?
Human: My favorite color is blue
AI: Blue is a great color! It's often associated with calmness, stability, and trustworthiness. Do you have a specific reason why blue is your favorite color?
Human: My favorite animals are dogs
AI:

> Finished chain.


{'input': 'My favorite animals are dogs',
 'history': "Human: Hi, my name is Julio\nAI: Hello, Julio! It's nice to meet you. How can I assist you today?\nHuman: My favorite color is blue\nAI: Blue is a great color! It's often associated with calmness, stability, and trustworthiness. Do you have a specific reason why blue is your favorite color?",
 'response': 'Dogs are wonderful animals! They are known for their loyalty, companionship, and playfulness. There are so many different breeds with unique characteristics. Do you have a favorite breed of dog?'}

In [20]:
conversation_window({"input": "I like to drive a vespa scooter in the city"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Julio
AI: Hello, Julio! It's nice to meet you. How can I assist you today?
Human: My favorite color is blue
AI: Blue is a great color! It's often associated with calmness, stability, and trustworthiness. Do you have a specific reason why blue is your favorite color?
Human: My favorite animals are dogs
AI: Dogs are wonderful animals! They are known for their loyalty, companionship, and playfulness. There are so many different breeds with unique characteristics. Do you have a favorite breed of dog?
Human: I like to drive a vespa scooter in the city
AI:

> Finished chain.


{'input': 'I like to drive a vespa scooter in the city',
 'history': "Human: Hi, my name is Julio\nAI: Hello, Julio! It's nice to meet you. How can I assist you today?\nHuman: My favorite color is blue\nAI: Blue is a great color! It's often associated with calmness, stability, and trustworthiness. Do you have a specific reason why blue is your favorite color?\nHuman: My favorite animals are dogs\nAI: Dogs are wonderful animals! They are known for their loyalty, companionship, and playfulness. There are so many different breeds with unique characteristics. Do you have a favorite breed of dog?",
 'response': 'Vespa scooters are a stylish and convenient way to navigate city streets! They have a classic design and are known for being fuel-efficient. Do you enjoy the feeling of riding a Vespa around the city?'}

In [21]:
conversation_window({"input": "My favorite city is San Francisco"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favorite color is blue
AI: Blue is a great color! It's often associated with calmness, stability, and trustworthiness. Do you have a specific reason why blue is your favorite color?
Human: My favorite animals are dogs
AI: Dogs are wonderful animals! They are known for their loyalty, companionship, and playfulness. There are so many different breeds with unique characteristics. Do you have a favorite breed of dog?
Human: I like to drive a vespa scooter in the city
AI: Vespa scooters are a stylish and convenient way to navigate city streets! They have a classic design and are known for being fuel-efficient. Do you enjoy the feeling of riding a Vespa around the 

{'input': 'My favorite city is San Francisco',
 'history': "Human: My favorite color is blue\nAI: Blue is a great color! It's often associated with calmness, stability, and trustworthiness. Do you have a specific reason why blue is your favorite color?\nHuman: My favorite animals are dogs\nAI: Dogs are wonderful animals! They are known for their loyalty, companionship, and playfulness. There are so many different breeds with unique characteristics. Do you have a favorite breed of dog?\nHuman: I like to drive a vespa scooter in the city\nAI: Vespa scooters are a stylish and convenient way to navigate city streets! They have a classic design and are known for being fuel-efficient. Do you enjoy the feeling of riding a Vespa around the city?",
 'response': "San Francisco is a vibrant city with a rich history and diverse culture! It's known for iconic landmarks like the Golden Gate Bridge, Alcatraz Island, and the historic cable cars. The city also has a reputation for its fantastic dining 

In [22]:
conversation_window({"input": "My favorite season is summer"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favorite animals are dogs
AI: Dogs are wonderful animals! They are known for their loyalty, companionship, and playfulness. There are so many different breeds with unique characteristics. Do you have a favorite breed of dog?
Human: I like to drive a vespa scooter in the city
AI: Vespa scooters are a stylish and convenient way to navigate city streets! They have a classic design and are known for being fuel-efficient. Do you enjoy the feeling of riding a Vespa around the city?
Human: My favorite city is San Francisco
AI: San Francisco is a vibrant city with a rich history and diverse culture! It's known for iconic landmarks like the Golden Gate Bridge, Alcatra

{'input': 'My favorite season is summer',
 'history': "Human: My favorite animals are dogs\nAI: Dogs are wonderful animals! They are known for their loyalty, companionship, and playfulness. There are so many different breeds with unique characteristics. Do you have a favorite breed of dog?\nHuman: I like to drive a vespa scooter in the city\nAI: Vespa scooters are a stylish and convenient way to navigate city streets! They have a classic design and are known for being fuel-efficient. Do you enjoy the feeling of riding a Vespa around the city?\nHuman: My favorite city is San Francisco\nAI: San Francisco is a vibrant city with a rich history and diverse culture! It's known for iconic landmarks like the Golden Gate Bridge, Alcatraz Island, and the historic cable cars. The city also has a reputation for its fantastic dining scene and beautiful views. What do you love most about San Francisco?",
 'response': "Summer is a popular season for many people because of the warm weather, longer day

In [23]:
conversation_window({"input": "What is my favorite color?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I like to drive a vespa scooter in the city
AI: Vespa scooters are a stylish and convenient way to navigate city streets! They have a classic design and are known for being fuel-efficient. Do you enjoy the feeling of riding a Vespa around the city?
Human: My favorite city is San Francisco
AI: San Francisco is a vibrant city with a rich history and diverse culture! It's known for iconic landmarks like the Golden Gate Bridge, Alcatraz Island, and the historic cable cars. The city also has a reputation for its fantastic dining scene and beautiful views. What do you love most about San Francisco?
Human: My favorite season is summer
AI: Summer is a popular season for

{'input': 'What is my favorite color?',
 'history': "Human: I like to drive a vespa scooter in the city\nAI: Vespa scooters are a stylish and convenient way to navigate city streets! They have a classic design and are known for being fuel-efficient. Do you enjoy the feeling of riding a Vespa around the city?\nHuman: My favorite city is San Francisco\nAI: San Francisco is a vibrant city with a rich history and diverse culture! It's known for iconic landmarks like the Golden Gate Bridge, Alcatraz Island, and the historic cable cars. The city also has a reputation for its fantastic dining scene and beautiful views. What do you love most about San Francisco?\nHuman: My favorite season is summer\nAI: Summer is a popular season for many people because of the warm weather, longer days, and opportunities for outdoor activities like swimming, barbecuing, and going to the beach. It's a great time for vacations and spending time with friends and family. Do you have any special summer traditions o

In [24]:
conversation_window({"input": "My favorite city is San Francisco"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favorite city is San Francisco
AI: San Francisco is a vibrant city with a rich history and diverse culture! It's known for iconic landmarks like the Golden Gate Bridge, Alcatraz Island, and the historic cable cars. The city also has a reputation for its fantastic dining scene and beautiful views. What do you love most about San Francisco?
Human: My favorite season is summer
AI: Summer is a popular season for many people because of the warm weather, longer days, and opportunities for outdoor activities like swimming, barbecuing, and going to the beach. It's a great time for vacations and spending time with friends and family. Do you have any special summer tra

{'input': 'My favorite city is San Francisco',
 'history': "Human: My favorite city is San Francisco\nAI: San Francisco is a vibrant city with a rich history and diverse culture! It's known for iconic landmarks like the Golden Gate Bridge, Alcatraz Island, and the historic cable cars. The city also has a reputation for its fantastic dining scene and beautiful views. What do you love most about San Francisco?\nHuman: My favorite season is summer\nAI: Summer is a popular season for many people because of the warm weather, longer days, and opportunities for outdoor activities like swimming, barbecuing, and going to the beach. It's a great time for vacations and spending time with friends and family. Do you have any special summer traditions or activities that you enjoy?\nHuman: What is my favorite color?\nAI: I'm sorry, I don't have access to that information. What is your favorite color?",
 'response': "San Francisco is a vibrant city with a rich history and diverse culture! It's known f

## Conversation Token Buffer Memory
Similar to the previous one, but this time **you can limit the number of tokens stored in memory**.

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [25]:
#!pip install tiktoken

In [26]:
from langchain.memory import ConversationTokenBufferMemory

In [27]:
token_memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=50
)

C:\Users\Juan\AppData\Local\Temp\ipykernel_19212\130807733.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  token_memory = ConversationTokenBufferMemory(


In [28]:
conversation_token = ConversationChain(
    llm=llm, 
    memory = token_memory,
    verbose=True
)

## Conversation Summary Memory
**Stores a summary** of the previous conversational exchanges.

In [29]:
from langchain.memory import ConversationSummaryBufferMemory

In [30]:
summary_memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100
)

C:\Users\Juan\AppData\Local\Temp\ipykernel_19212\3597302779.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryBufferMemory(


In [31]:
conversation_summary = ConversationChain(
    llm=llm, 
    memory = summary_memory,
    verbose=True
)

In [32]:
conversation_summary({"input": """Kurt Cobain dropped out of high school, 
then worked there as a janitor Even though he was by all accounts a slob, 
Kurt Cobain worked as a janitor at Weatherwax High School, not long after 
dropping out of that very school. The dancing janitor in the 
"Smells Like Teen Spirit" music video was an inside joke for 
those who knew of Cobain's old job.
"""})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Kurt Cobain dropped out of high school, 
then worked there as a janitor Even though he was by all accounts a slob, 
Kurt Cobain worked as a janitor at Weatherwax High School, not long after 
dropping out of that very school. The dancing janitor in the 
"Smells Like Teen Spirit" music video was an inside joke for 
those who knew of Cobain's old job.

AI:

> Finished chain.


{'input': 'Kurt Cobain dropped out of high school, \nthen worked there as a janitor Even though he was by all accounts a slob, \nKurt Cobain worked as a janitor at Weatherwax High School, not long after \ndropping out of that very school. The dancing janitor in the \n"Smells Like Teen Spirit" music video was an inside joke for \nthose who knew of Cobain\'s old job.\n',
 'history': '',
 'response': 'Wow, that\'s some interesting trivia about Kurt Cobain! It\'s always fascinating to learn little details about famous people\'s lives. Did you know that the janitor character in the "Smells Like Teen Spirit" music video was actually played by actor Tony De La Rosa, and not Kurt Cobain himself? It was definitely a clever nod to Cobain\'s past as a janitor at Weatherwax High School. It\'s amazing how these little connections can add so much depth to a music video. If you have any more questions or facts to share, feel free to let me know!'}

In [33]:
conversation_summary({"input": """
There were at least five different drummers in the band 
before Dave Grohl. Cobain and Novoselic were always members 
of Nirvana—formerly known as Skid Row, Pen Cap Chew, Bliss, 
and Ted Ed Fred—but finding a permanent drummer proved to be 
even harder than coming up with a decent band name. In the 
beginning, there was trivia answer Aaron Burckhard, who pissed 
off Cobain by getting Kurt's car impounded after being arrested 
for fighting with a police officer. Then there was Melvins 
drummer Dale Crover, who pounded the skins for Cobain and 
Novoselic on their first demo tape before moving to San Francisco. 
Next came Dave Foster, who got arrested for assaulting the 
son of the mayor of Cosmopolis, Washington. Burckhard briefly 
returned before announcing he was too hungover to practice one day. 
Then a mutual friend introduced Cobain and Novoselic to Chad 
Channing, who hung around for two years before the group's 
co-founders decided he wasn't cutting it anymore. Mudhoney 
drummer Dan Peters played on the "Sliver" single.
"""})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human shares a piece of trivia about Kurt Cobain working as a janitor at Weatherwax High School after dropping out. The AI finds this interesting and expands on the fact by mentioning that the janitor character in the "Smells Like Teen Spirit" music video was played by actor Tony De La Rosa. The AI appreciates how these details add depth to the music video and invites more questions and facts to be shared.
Human: 
There were at least five different drummers in the band 
before Dave Grohl. Cobain and Novoselic were always members 
of Nirvana—formerly known as Skid Row, Pen Cap Chew, Bliss, 
and Ted Ed Fred—but finding a permanent drummer proved to be 
even h

{'input': '\nThere were at least five different drummers in the band \nbefore Dave Grohl. Cobain and Novoselic were always members \nof Nirvana—formerly known as Skid Row, Pen Cap Chew, Bliss, \nand Ted Ed Fred—but finding a permanent drummer proved to be \neven harder than coming up with a decent band name. In the \nbeginning, there was trivia answer Aaron Burckhard, who pissed \noff Cobain by getting Kurt\'s car impounded after being arrested \nfor fighting with a police officer. Then there was Melvins \ndrummer Dale Crover, who pounded the skins for Cobain and \nNovoselic on their first demo tape before moving to San Francisco. \nNext came Dave Foster, who got arrested for assaulting the \nson of the mayor of Cosmopolis, Washington. Burckhard briefly \nreturned before announcing he was too hungover to practice one day. \nThen a mutual friend introduced Cobain and Novoselic to Chad \nChanning, who hung around for two years before the group\'s \nco-founders decided he wasn\'t cutting 

In [34]:
conversation_summary({"input": """
Back in Washington, Crover performed with Cobain and Novoselic 
on a seven date tour with Sonic Youth in August 1990, before 
Dave Grohl's band Scream broke up and Melvins frontman Buzz 
Osbourne introduced Grohl to Cobain and Novoselic, ending the 
vicious cycle of rotating drummers.
"""})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human shares a piece of trivia about Kurt Cobain working as a janitor at Weatherwax High School after dropping out. The AI finds this interesting and expands on the fact by mentioning that the janitor character in the "Smells Like Teen Spirit" music video was played by actor Tony De La Rosa. The AI appreciates how these details add depth to the music video and invites more questions and facts to be shared, like the history of Nirvana's drummers before Dave Grohl.
AI: Wow, that's quite a history of drummers for Nirvana before Dave Grohl! It's interesting to see how the lineup changes shaped the band's sound and direction. Each drummer probably brought someth

{'input': "\nBack in Washington, Crover performed with Cobain and Novoselic \non a seven date tour with Sonic Youth in August 1990, before \nDave Grohl's band Scream broke up and Melvins frontman Buzz \nOsbourne introduced Grohl to Cobain and Novoselic, ending the \nvicious cycle of rotating drummers.\n",
 'history': 'System: The human shares a piece of trivia about Kurt Cobain working as a janitor at Weatherwax High School after dropping out. The AI finds this interesting and expands on the fact by mentioning that the janitor character in the "Smells Like Teen Spirit" music video was played by actor Tony De La Rosa. The AI appreciates how these details add depth to the music video and invites more questions and facts to be shared, like the history of Nirvana\'s drummers before Dave Grohl.\nAI: Wow, that\'s quite a history of drummers for Nirvana before Dave Grohl! It\'s interesting to see how the lineup changes shaped the band\'s sound and direction. Each drummer probably brought some

In [35]:
print(summary_memory.buffer)

System: The human shares a piece of trivia about Kurt Cobain working as a janitor at Weatherwax High School after dropping out. The AI finds this interesting and expands on the fact by mentioning that the janitor character in the "Smells Like Teen Spirit" music video was played by actor Tony De La Rosa. The AI appreciates how these details add depth to the music video and invites more questions and facts to be shared, like the history of Nirvana's drummers before Dave Grohl, discussing how each drummer contributed uniquely to the band's sound. The human shares more trivia about Crover performing with Cobain and Novoselic before the introduction of Dave Grohl, highlighting the challenges in finding the right fit for Nirvana's lineup.
AI: That's an interesting piece of trivia! It's amazing how interconnected the music scene was back then, with all these bands collaborating and influencing each other. It seems like fate brought Dave Grohl to Nirvana at just the right time. It's fascinatin

In [36]:
summary_memory.load_memory_variables({})

{'history': 'System: The human shares a piece of trivia about Kurt Cobain working as a janitor at Weatherwax High School after dropping out. The AI finds this interesting and expands on the fact by mentioning that the janitor character in the "Smells Like Teen Spirit" music video was played by actor Tony De La Rosa. The AI appreciates how these details add depth to the music video and invites more questions and facts to be shared, like the history of Nirvana\'s drummers before Dave Grohl, discussing how each drummer contributed uniquely to the band\'s sound. The human shares more trivia about Crover performing with Cobain and Novoselic before the introduction of Dave Grohl, highlighting the challenges in finding the right fit for Nirvana\'s lineup.\nAI: That\'s an interesting piece of trivia! It\'s amazing how interconnected the music scene was back then, with all these bands collaborating and influencing each other. It seems like fate brought Dave Grohl to Nirvana at just the right ti